크롤링

In [2]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들
from time import sleep

In [10]:
import sys
##네이버 블로그 글
def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)


options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(ChromeDriverManager().install())
# driver = webdriver.Chrome(options=options)
 
# 대기 시간
driver.implicitly_wait(time_to_wait=3)
 
# 반복 종료 조건
loop = True
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://map.naver.com/v5/") #네이버지도 열기
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

keyword = '서울 중구 카페'

search_box = driver.find_element(By.CLASS_NAME,"input_search") #검색창 찾기
search_box.send_keys(keyword) #검색어 입력
search_box.send_keys(Keys.ENTER) #검색
sleep(5)
while(True):
    switch_left()
 
    # 이건 페이지 넘어갈때마다 확인 -> true라면 마지막 페이지임
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[7]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break
    ##스크롤 내리기##
    a=0
    while a<15:
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        driver.execute_script("arguments[0].scrollTop += 500;", scrollable_element)##변경필요
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        a+=1

    """
    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
 
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
 
    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
 
        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절
 
        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
 
        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break
 
        last_height = new_height
 
    """

    ############## 현재 page number 가져오기 - 1 페이지 ##############
 
    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text
 
    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li')
 
    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')
 
    for index, e in enumerate(elemets, start=1):
        final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span")
        final_element = e.find_element(By.XPATH, "./div[1]/a/div/div/span[1]")
        print(str(index) + ". " + final_element.text)
 
    #print(Colors.RED + "-"*50 + Colors.RESET)
 
    switch_left()
 
    sleep(2)
 
    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름
        category = '' # 카테고리
        #new_open = '' # 새로 오픈
        
        #rating = 0.0 # 평점
        #visited_review = 0 # 방문자 리뷰
        #blog_review = 0 # 블로그 리뷰
        #store_id = '' # 가게 고유 번호
        
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호
 
        switch_left()
 
 
        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()
 
        sleep(2)
 
        switch_right()
 
        ################### 여기부터 크롤링 시작 ##################
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        # store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')
 
        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text
 
        # if(len(store_info) > 2):
        #     # 새로 오픈
        #     new_open = title.find_element(By.XPATH,'.//div[1]/div[1]/span[3]').text
 
        ###############################
 
        # review = title.find_elements(By.XPATH,'.//div[2]/span')
 
        # # 인덱스 변수 값
        # _index = 1
 
        # # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        # if len(review) > 2:
        #     rating_xpath = f'.//div[2]/span[{_index}]'
        #     rating_element = title.find_element(By.XPATH, rating_xpath)
        #     rating = rating_element.text.replace("\n", " ")
 
        #     _index += 1
 
        # try:
        #     # 방문자 리뷰
        #     visited_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text
 
 
        #     # 인덱스를 다시 +1 증가 시킴
        #     _index += 1
 
        #     # 블로그 리뷰
        #     blog_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text
        # except:
        #     print(Colors.RED + '------------ 리뷰 부분 오류 ------------' + Colors.RESET)
 
        # # 가게 id
        # store_id = driver.find_element(By.XPATH,'//div[@class="flicking-camera"]/a').get_attribute('href').split('/')[4]
 
 
        #전화번호
        phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text
        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text
 
        print(f'가게명: {store_name},   업종: {category},   전화번호: {phone_num},   주소: {address}')

        # try:
        #     driver.find_element(By.XPATH,'//div[@class="y6tNq"]//span').click()
 
        #     # 영업 시간 더보기 버튼을 누르고 2초 반영시간 기다림
        #     sleep(2)
 
 
        #     parent_element = driver.find_element(By.XPATH,'//a[@class="gKP9i RMgN0"]')
        #     child_elements = parent_element.find_elements(By.XPATH, './*[@class="w9QyJ" or @class="w9QyJ undefined"]')
 
        #     for child in child_elements:
        #         # 각 자식 요소 내에서 클래스가 'A_cdD'인 span 요소 찾기
        #         span_elements = child.find_elements(By.XPATH, './/span[@class="A_cdD"]')
 
        #         # 찾은 span 요소들의 텍스트 출력
        #         for span in span_elements:
        #             business_hours.append(span)
            
        #     # 가게 전화번호
        #     phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text
 
        # except:
        #     print(print(Colors.RED + '------------ 영업시간 / 전화번호 부분 오류 ------------' + Colors.RESET))
        
        # print(Colors.BLUE + f'{index}. ' + str(store_name) + Colors.RESET + ' · ' + str(category) + Colors.RED + str(new_open) + Colors.RESET)
        # print('평점 ' + Colors.RED + str(rating) + Colors.RESET + ' / ' + visited_review + ' · ' + blog_review)
        # print(f'가게 고유 번호 -> {store_id}')
        # print('가게 주소 ' + Colors.GREEN + str(address) + Colors.RESET)
        # print(Colors.CYAN + '가게 영업 시간' + Colors.RESET)
        # for i in business_hours:
        #     print(i.text)
        #     print('')
        # print('가게 번호 ' + Colors.GREEN + phone_num + Colors.RESET)
        # print(Colors.MAGENTA + "-"*50 + Colors.RESET)
    
    switch_left()
        
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').click()
    # 아닐 경우 루프 정지
    else:
        loop = False


현재 1 페이지 / 총 28개의 가게를 찾았습니다.

1. 크림시크
2. 베카롱베이글
3. 더 스팟 패뷸러스
4. 공간갑
5. 덕수궁 리에제와플
6. 커피앤시가렛
7. 태극당
8. 커피한약방
9. 분카샤 을지로점
10. 적당
11. 아소토베이커리
12. 맷차 명동본점
13. 빠우 을지로점
14. 브레디포스트 명동점
15. 르빵 명동성당점
16. 섹터커피 충무로점
17. 리사르커피 약수점
18. 피즈소셜클럽
19. 커피스니퍼 서울시청점
20. 몰또 이탈리안 에스프레소바
21. 베러먼데이커피 서울역센트럴점
22. 덕수궁 카페 돌담콩
23. 어니스트
24. 섹터커피 을지로점
25. 재해석
26. 포엠
27. 수잔나의 앞치마
28. 커피루소 정동
가게명: 크림시크,   업종: 카페,   전화번호: 0507-1351-9271,   주소: 서울 중구 명동7길 13 1층 크림시크
가게명: 베카롱베이글,   업종: 베이글,   전화번호: 010-7556-2975,   주소: 서울 중구 다산로42길 49 101호
가게명: 더 스팟 패뷸러스,   업종: 카페,디저트,   전화번호: 0507-1422-1981,   주소: 서울 중구 명동2길 22 1.2층
가게명: 공간갑,   업종: 카페,디저트,   전화번호: 0507-1362-4569,   주소: 서울 중구 수표로 48-8 1, 2층
가게명: 덕수궁 리에제와플,   업종: 와플,   전화번호: 0507-1339-5202,   주소: 서울 중구 덕수궁길 5
가게명: 커피앤시가렛,   업종: 카페,   전화번호: 02-777-7557,   주소: 서울 중구 서소문로 116 유원빌딩17층 1706호
가게명: 태극당,   업종: 베이커리,   전화번호: 02-2279-3152,   주소: 서울 중구 동호로24길 7
가게명: 커피한약방,   업종: 카페,   전화번호: 070-4148-4242,   주소: 서울 중구 삼일대로12길 16-6
가게명: 분카샤 을지로점,   업종: 카페,디저트,   전화번호: 0507-1401-6086,  

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div[3]/div[2]/a[7]"}
  (Session info: chrome=122.0.6261.113)


In [53]:
#가게 1개 - class="UEzoS rTjJo"
from multiprocessing import cpu_count
print("최대 CPU 개수는 %s 입니다."%cpu_count())

최대 CPU 개수는 16 입니다.
